In [1]:
import sys
from pathlib import Path

# Add project root to sys.path so `src/` can be imported
sys.path.append(str(Path("..").resolve()))


In [3]:
import pandas as pd
import joblib
import hopsworks
from datetime import datetime, timedelta
from pathlib import Path

from src.config import (
    HOPSWORKS_API_KEY,
    HOPSWORKS_PROJECT_NAME,
    FEATURE_VIEW_NAME,
    FEATURE_VIEW_VERSION,
    MODELS_DIR
)
from src.data_utils import transform_ts_data_info_features


In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project=HOPSWORKS_PROJECT_NAME)
fs = project.get_feature_store()
fv = fs.get_feature_view(name=FEATURE_VIEW_NAME, version=FEATURE_VIEW_VERSION)

predict_for_hour = pd.Timestamp.now(tz="UTC").ceil("h")
start_time = predict_for_hour - timedelta(days=28)
end_time = predict_for_hour - timedelta(hours=1)

ts_data = fv.get_batch_data(start_time=start_time, end_time=end_time)
ts_data = ts_data[ts_data["start_hour"].between(start_time, end_time)].sort_values(["start_station_id", "start_hour"]).reset_index(drop=True)
ts_data["start_hour"] = pd.to_datetime(ts_data["start_hour"])

ts_data.tail()


2025-05-10 03:15:19,286 INFO: Closing external client and cleaning up certificates.
2025-05-10 03:15:19,289 INFO: Initializing external client
2025-05-10 03:15:19,289 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-10 03:15:20,762 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228959
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.68s) 


,start_hour,start_station_id,rides,hour,day_of_week,is_weekend,month,date


In [10]:
print(ts_data.shape)


(0, 8)
